<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Variable Test Base </span> </p>

In [1]:
#!import "../Report/ReportScopes"

# Helpers

In [1]:
const string benchmarkFileNamePrefix = "ReportVariableBenchmarks_";

In [1]:
static string GetBenchmarkFileName(((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) args) => args.ToString().Replace("(", "").Replace(")", "").Replace(" ", "").Replace(",", "_");

# Calculated

In [1]:
static ICollection<ReportVariable> GetScopesData(Systemorph.Vertex.Scopes.IScopeWithStorage<ReportStorage> universe, ICollection<(ReportIdentity, CurrencyType)> identities) {
    
    var lockedBePvs = universe.GetScopes<LockedBestEstimate>(identities).Aggregate().LockedBestEstimate;
    var currentBePvs = universe.GetScopes<CurrentBestEstimate>(identities).Aggregate().CurrentBestEstimate;       

    var lockedRaPvs = universe.GetScopes<LockedRiskAdjustment>(identities).Aggregate().LockedRiskAdjustment;
    var currentRaPvs = universe.GetScopes<CurrentRiskAdjustment>(identities).Aggregate().CurrentRiskAdjustment;

    var writtenActuals = universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Written;
    var advanceActuals = universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Advance;
    var overdueActuals = universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Overdue;

    var deferrableActuals = universe.GetScopes<Deferrals>(identities).Aggregate().Deferrals;

    var csm = universe.GetScopes<Csm>(identities).Aggregate().Csm;
    var lc = universe.GetScopes<Lc>(identities).Aggregate().Lc;
    var loreco = universe.GetScopes<Loreco>(identities).Aggregate().Loreco;

    var fp = universe.GetScopes<FinancialPerformance>(identities).Aggregate().FinancialPerformance;
    
    return Enumerable.Empty<ReportVariable>()
    .Concat(lockedBePvs)    
    .Concat(currentBePvs)
    .Concat(lockedRaPvs)
    .Concat(currentRaPvs)
    .Concat(writtenActuals)
    .Concat(advanceActuals)
    .Concat(overdueActuals)
    .Concat(deferrableActuals)
    .Concat(csm)
    .Concat(lc)
    .Concat(loreco)
    .Concat(fp)
    .OrderBy(v => v.GroupOfContract).ThenBy(v => v.Scenario).ThenBy(v => v.Projection).ThenBy(v => v.AccidentYear).ThenBy(v => v.EstimateType).ThenBy(v => (v.VariableType, v.Novelty))
    .ToArray();
}

In [1]:
static async Task<ICollection<ReportVariable>> GetScopesDataAsync(((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) args, ReportStorage reportStorage, Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes ) {
    await reportStorage.InitializeAsync(args.Period, args.ReportingNode, args.Scenario, args.CurrencyType);
    var identities = reportStorage.GetIdentities(args.Period, args.ReportingNode, args.Scenario, args.CurrencyType);
    var universe = scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();
    return GetScopesData(universe, identities);
}

# Benchmarks

In [1]:
static async Task<ICollection<ReportVariable>> GetBenchmarkDataAsync(((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) args, string path, IWorkspace workspace, IImportVariable importVariable) {
    var bmFileName = benchmarkFileNamePrefix + GetBenchmarkFileName(args) + ".csv";
    await workspace.Partition.SetAsync<PartitionByReportingNode>(new { ReportingNode = args.ReportingNode, Scenario = args.Scenario });
    await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new { ReportingNode = args.ReportingNode, Scenario = args.Scenario, Year = args.Period.Year, Month = args.Period.Month });
    await importVariable.FromFile(path + bmFileName).WithType<ReportVariable>(x => x.SnapshotMode()).WithTarget(workspace).ExecuteAsync();
    var ret = await workspace.Query<ReportVariable>().ToArrayAsync();
    workspace.Reset(x => x.ResetCurrentPartitions().ResetType<ReportVariable>());
    return ret;
}